### Imports

In [30]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

%run utils.ipynb

### Model building

In [31]:
class RNN(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim = 200, hidden_size = 256, num_layers=3):
        """
        The constructor of our RNN model
        Inputs:
        - vacab_size: the number of unique characters
        - embedding_dim: the embedding dimension
        - n_classes: the number of final classes (diacritics)
        """
        super(RNN, self).__init__()

        # (1) Create an embedding layer
        # self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # (3) Create a linear layer with number of neorons = n_classes
        self.linear = nn.Linear(hidden_size*2, n_classes)

    def forward(self, sentences):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        final_output = None
        
        # embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(sentences)
        output = self.linear(lstm_out)
        final_output = F.softmax(output, dim=1)
        return final_output
    

class RNN_FastText(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim = 200, hidden_size = 256, num_layers=3):
        """
        The constructor of our RNN model
        Inputs:
        - vacab_size: the number of unique characters
        - embedding_dim: the embedding dimension
        - n_classes: the number of final classes (diacritics)
        """
        super(RNN, self).__init__()

        # Load FastText model
        from gensim.models import FastText
        fasttext_model = FastText.load("./models/ft_model")

        # Extract word embeddings from FastText model
        words = fasttext_model.wv.index_to_key
        weights = [fasttext_model.wv.get_vector(word) for word in words]
        pre_trained_embeddings = torch.tensor(weights)

        # (1) Create the embedding layer and load pre-trained word embeddings
        self.embedding = nn.Embedding.from_pretrained(pre_trained_embeddings, freeze=True)

        # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # (3) Create a linear layer with number of neorons = n_classes
        self.linear = nn.Linear(hidden_size*2, n_classes)

    def forward(self, sentences):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        final_output = None
        
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        final_output = F.softmax(output, dim=1)
        return final_output

### Train

In [32]:
def train(model, train_dataset, train_labels, batch_size=512, epochs=20, learning_rate=0.001):
    """
    This function implements the training logic
    Inputs:
    - model: the model to be trained
    - train_dataset: the training set
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    # (1) create the dataloader of the training set (make the shuffle=True)
    tensor_train_dataset = TensorDataset(train_dataset, train_labels)
    train_dataloader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()

    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    best_accuracy = 0.0
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        
        for train_input, train_label in tqdm(train_dataloader):

            # (4) move the train input to the device
            train_label = train_label.to(device)

            # (5) move the train label to the device
            train_input = train_input.to(device)

            # (6) do the forward pass
            output = model(train_input).float()

            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            one_hot_train_label = F.one_hot(train_label, num_classes=15).float()
            batch_loss = criterion(output, one_hot_train_label)

            # (8) append the batch loss to the total_loss_train
            total_loss_train += batch_loss
            
            # (9) calculate the batch accuracy (just add the number of correct predictions)
            # Compare predicted diacritic with true diacritic and count correct predictions
            correct_predictions = (output.argmax(dim=2) == train_label)

            # Calculate accuracy for the current batch
            acc = correct_predictions.sum().item()

            total_acc_train += acc

            # (10) zero your gradients
            optimizer.zero_grad()

            # (11) do the backward pass
            batch_loss.backward()

            # (12) update the weights with your optimizer
            optimizer.step()     
        
        # epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # (13) calculate the accuracy
        epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0]))
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')
        if epoch_acc > best_accuracy:
            best_accuracy = epoch_acc
            torch.save(model.state_dict(), LSTM_PATH)
            print(f'Saved the best model with accuracy: {best_accuracy} to {LSTM_PATH}')


In [33]:
corpus=  readFile(TRAIN_PATH)

x_train = []
y_train = []

for sentence in corpus[:100]:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

	x_train.append(char_list)
	y_train.append(diacritics_list)

X_train_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in x_train for word in sentence]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_train for word in sentence]
print(y_train_padded)
y_train_padded = pad_sequence(y_train_padded, batch_first=True)
print(y_train_padded)

[tensor([1, 7, 3, 3]), tensor([1, 7]), tensor([1, 1, 1]), tensor([0, 7, 1, 9, 3]), tensor([1, 1, 3]), tensor([0, 1, 7]), tensor([1, 0, 1]), tensor([ 0,  0,  9,  7,  1,  5, 11]), tensor([0, 7, 3]), tensor([1, 1, 1, 1]), tensor([1, 7, 3, 3]), tensor([5, 1, 7, 6]), tensor([1, 7, 1, 5, 0, 0]), tensor([1, 5, 7, 1, 0, 5]), tensor([1, 7, 5]), tensor([1, 5, 0, 6]), tensor([5, 0, 7, 5, 7, 1, 0, 5]), tensor([3, 3, 0, 1]), tensor([1, 0]), tensor([3, 5, 1]), tensor([3, 3, 0, 3, 3]), tensor([5, 7]), tensor([ 0,  0, 13,  0,  5]), tensor([1, 3, 0, 1, 2]), tensor([1, 5, 7, 1, 0, 5]), tensor([3, 7, 1, 6]), tensor([5, 1, 1, 6]), tensor([ 1,  1, 13]), tensor([3, 9, 0, 6]), tensor([0, 7, 3]), tensor([1, 1, 1, 1]), tensor([1, 7, 3]), tensor([0, 7, 5]), tensor([1, 0, 6]), tensor([1, 7]), tensor([5, 5, 7, 6]), tensor([1, 1, 1, 9, 3, 3]), tensor([3, 1]), tensor([1, 3, 7, 5]), tensor([ 0,  0, 11,  9,  0,  5]), tensor([1, 5, 7, 1, 0, 5]), tensor([0, 7, 3, 7, 1, 5]), tensor([5, 0]), tensor([1, 5, 0, 5]), tensor(

In [34]:
def run():
    model=RNN(len(unique_characters) + 1, len(unique_diacritics), embedding_dim=200, hidden_size=256, num_layers=3)
    print(model)
    train(model, X_train_padded, y_train_padded, batch_size=512, epochs=10, learning_rate=0.1)

In [35]:
run()

RNN(
  (lstm): LSTM(200, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=15, bias=True)
)


  0%|          | 0/9 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float